In [1]:
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import os

data_path = os.path.join('datasets', 'open-data')

In [2]:
def generate_days_fn(init_date, final_date):
    
    delta = final_date - init_date       # as timedelta

    target_days = []
    for i in range(delta.days + 1):
        day = init_date + datetime.timedelta(days=i)
        target_days.append(day)
    
    return target_days

def read_INE_trips_date_fn(date_, trips_type='all'):
    date_str= date_.strftime('%Y%m%d')
    df_date= pd.read_csv(os.path.join(data_path, f'{date_str}_maestra_1_mitma_distrito.txt'), 
                             sep='|',dtype={'origen':str, 'destino':str,'fecha':str, 'periodo':str})
    
    if trips_type=='inter':
        df_date= df_date[df_date['origen']!=df_date['destino']] #only keep trips between areas
    elif trips_type=='intra':
        df_date= df_date[df_date['origen']==df_date['destino']] #only keep trips within the areas
    
    #convert period column to a two-digit string
    df_date['periodo'] = df_date['periodo'].apply(lambda x: x.zfill(2))
    df_date= df_date.fillna(0) # set nan as 0
    return df_date

def generate_hourly_time_series_fn(from_date, to_date, target_col, trips_type='all'):
     
    time_series = []
    target_days = generate_days_fn(from_date, to_date)
    for date in tqdm(target_days):
        df = read_INE_trips_date_fn(date, trips_type)
        df= df.drop(['edad', 'viajes_km','residencia'], axis=1)
        #print(df)
        group_by_df= df.groupby(['fecha',target_col,'periodo'])['viajes'].sum()
        group_by_df= group_by_df.reset_index()
        group_by_df['fecha_periodo'] = group_by_df['fecha']+'_'+ group_by_df['periodo'].astype(str)
        group_by_df= group_by_df.drop('fecha periodo'.split(), axis=1)
        total_trips_final = group_by_df.pivot_table(values='viajes',index='fecha_periodo',columns=target_col)
        time_series.append(total_trips_final)
    ts_df= pd.concat(time_series, axis=0)
    ts_df.index= ts_df.index.map(lambda x: datetime.datetime.strptime(x, "%Y%m%d_%H"))
    ts_df = ts_df.fillna(0.0)
    return ts_df

In [3]:
end_date = datetime.datetime.strptime('2021-02-01', '%Y-%m-%d')
start_date = datetime.datetime.strptime('2021-01-01', '%Y-%m-%d')

In [38]:
out_ts= generate_hourly_time_series_fn(start_date, end_date, 'origen')

/Users/Fer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


A Jupyter Widget

In [39]:
out_ts.head()

,01001_AM,01002,01010_AM,01031_AM,01036,01043_AM,01047_AM,01051,01058_AM,0105901,...,5100104,5100105,5100106,5200101_AD,5200102,5200104,5200105,5200106,5200107,5200108
fecha_periodo,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,704.495,891.841,395.750,364.283,1017.091,401.675,148.055,162.104,413.252,3590.050,...,2509.446,645.169,1756.574,559.811,946.718,940.340,1831.866,1136.832,2101.492,4545.376
2021-01-01 01:00:00,475.394,383.715,103.368,134.591,451.060,82.742,111.488,108.708,165.096,1514.325,...,1720.541,707.977,1177.365,340.359,369.506,562.083,996.524,563.260,995.418,2007.362
2021-01-01 02:00:00,294.631,250.428,74.897,153.053,427.143,109.860,138.339,0.000,146.667,825.691,...,924.498,437.257,571.404,255.229,284.748,323.358,542.062,291.590,466.025,1075.579
2021-01-01 03:00:00,174.496,252.497,33.494,37.952,127.404,95.856,20.728,31.222,118.945,984.173,...,597.566,301.482,489.295,131.313,205.985,207.657,472.578,234.656,283.300,944.633
2021-01-01 04:00:00,105.544,145.848,22.884,161.906,163.156,12.841,80.056,0.000,69.758,683.601,...,436.441,259.594,383.955,138.510,54.902,154.508,351.339,146.658,299.981,720.424


In [40]:
out_ts.tail()

,01001_AM,01002,01010_AM,01031_AM,01036,01043_AM,01047_AM,01051,01058_AM,0105901,...,5100104,5100105,5100106,5200101_AD,5200102,5200104,5200105,5200106,5200107,5200108
fecha_periodo,,,,,,,,,,,,,,,,,,,,,
2021-02-01 19:00:00,765.041,1036.875,410.095,414.703,1623.306,573.588,231.707,235.636,647.300,5126.597,...,3466.355,1124.351,1873.280,931.692,1035.554,1066.690,1550.205,1435.672,2507.785,4572.755
2021-02-01 20:00:00,629.453,898.349,237.660,329.487,1366.298,263.384,228.845,302.165,529.233,4615.871,...,2628.723,890.359,1837.760,650.917,963.530,917.181,1306.524,1024.275,1461.679,3328.900
2021-02-01 21:00:00,713.701,705.160,241.425,348.062,1124.714,311.107,300.458,140.631,521.337,3404.407,...,2389.625,891.729,1759.009,397.523,572.101,1029.823,1228.703,819.761,1176.595,3483.820
2021-02-01 22:00:00,486.330,515.382,204.280,306.617,799.817,295.218,215.796,85.539,603.049,2019.384,...,1496.753,652.617,1294.086,324.435,399.815,415.655,823.652,607.471,781.959,2087.357
2021-02-01 23:00:00,320.662,258.336,64.092,310.089,408.159,118.473,103.764,58.698,206.334,1094.579,...,900.668,341.862,627.233,237.180,301.659,398.572,566.318,330.258,686.802,1310.705


In [41]:
out_ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 768 entries, 2021-01-01 00:00:00 to 2021-02-01 23:00:00
Columns: 2841 entries, 01001_AM to 5200108
dtypes: float64(2841)
memory usage: 16.7 MB


In [ ]:
out_ts.to_csv(os.path.join('datasets', 'open-data', 'raw_open_data_outgoing.csv'))

In [4]:
in_ts= generate_hourly_time_series_fn(start_date, end_date, 'destino')
in_ts.head()

  0%|          | 0/32 [00:00<?, ?it/s]

destino,01001_AM,01002,01010_AM,01031_AM,01036,01043_AM,01047_AM,01051,01058_AM,0105901,...,5200105,5200106,5200107,5200108,09048_AM,20005_AM,20052_AM,02067_AM,24183_AM,20903
fecha_periodo,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,813.552,764.277,402.175,353.913,1095.245,383.888,232.899,126.621,488.064,3086.384,...,1716.988,992.061,2247.142,4447.699,0.0,0.0,0.0,0.0,0.0,0.0
2021-01-01 01:00:00,365.676,448.945,105.656,162.818,540.955,166.128,105.132,96.908,161.482,1324.430,...,845.684,554.854,903.086,2288.906,0.0,0.0,0.0,0.0,0.0,0.0
2021-01-01 02:00:00,214.170,196.794,84.374,143.784,369.640,103.738,88.993,101.501,182.207,1005.258,...,458.633,263.918,485.711,1193.421,0.0,0.0,0.0,0.0,0.0,0.0
2021-01-01 03:00:00,200.419,213.323,22.884,83.698,176.347,43.390,17.790,0.000,75.474,894.974,...,446.601,244.417,326.795,789.207,0.0,0.0,0.0,0.0,0.0,0.0
2021-01-01 04:00:00,100.942,116.751,83.699,102.332,196.584,55.151,116.476,29.226,118.473,752.135,...,180.933,215.591,275.610,701.698,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
in_ts.to_csv(os.path.join('datasets', 'open-data', 'raw_open_data_incoming.csv'))

In [6]:
print("That's all folks!")

That's all folks!
